In [429]:
import polars as pl
import matplotlib.pyplot as plt
import pathlib


In [430]:
ruta = pathlib.Path().parent.resolve()
data = f"{ruta}/data/sapal_salarios_clean.csv"


salarios_sapal =(
    pl.scan_csv(data)
)

df_filtrado = (
    salarios_sapal
    .select([
        pl.col("denominacion_del_cargo").alias("cargo"),
        pl.col("monto_mensual_bruto_de_la_remuneracion_en_tabulador").alias("salario_bruto_mensual")
    ])

)



df_cargos_unicos = (
    df_filtrado
    .select(
        pl.col("cargo")
    )
    .unique()
    .sort("cargo")
    .collect()
    .get_column("cargo")
    .to_list()
)


df_filtrado.collect()


cargo,salario_bruto_mensual
str,f64
"""SECRETARIA""",10416.6
"""PROGRAMADORA ANALISTA ""A""""",16699.0
"""SUP. CATASTRO FISICO INST.""",19532.3
"""AUX. ADMINISTRATIVA REDES""",12696.0
"""EJECUTIVA DEL SISTEMA DE ATENC…",15239.8
…,…
"""CHOFER DE TOLVA""",13921.1
"""CHOFER DE TOLVA""",13921.1
"""VIGILANTE DE POZOS""",13653.9


In [431]:


lf = (
    pl.scan_csv("data/sapal_salarios_clean.csv")
    .select([
        pl.col("denominacion_del_cargo").alias("cargo"),
        pl.col("monto_mensual_bruto_de_la_remuneracion_en_tabulador")
          .alias("salario_bruto_mensual"),
    ])
    .filter(
        pl.col("cargo").is_not_null() & (pl.col("cargo") != "")
    )
    .filter(
        pl.col("salario_bruto_mensual").is_not_null()
    )
)

lf_resumen_cargos = (
    lf
    .group_by("cargo")
    .agg([
        pl.len().alias("num_personas_en_cargo"),
        pl.col("salario_bruto_mensual").mean().alias("salario_bruto_promedio"),
        pl.col("salario_bruto_mensual").max().alias("salario_bruto_max"),
        pl.col("salario_bruto_mensual").sum().alias("costo_bruto_total_mensual_cargo"),
    ])
    .sort("salario_bruto_promedio", descending=True)
)

# ahora sí ejecutamos:
df_resumen_cargos = lf_resumen_cargos.collect()
df_resumen_cargos

cargo,num_personas_en_cargo,salario_bruto_promedio,salario_bruto_max,costo_bruto_total_mensual_cargo
str,u32,f64,f64,f64
"""DIRECTOR GENERAL""",1,141667.3,141667.3,141667.3
"""SUBDIRECTOR GENERAL OPERATIVO""",1,101905.4,101905.4,101905.4
"""SUBDIRECTORA DE PLANEACION""",1,90119.6,90119.6,90119.6
"""JEFE DE JURIDICO""",1,86502.0,86502.0,86502.0
"""PROSECRETARIO""",1,86502.0,86502.0,86502.0
…,…,…,…,…
"""EVENTUAL""",112,9800.0,9800.0,1.0976e6
"""PEÓN DE MANTENIMIENTO SERVICIO…",8,9800.0,9800.0,78400.0
"""PEON ALCANTARILLADO""",24,9800.0,9800.0,235200.0


In [432]:
print(f"Total de trabajadores: {df_resumen_cargos['num_personas_en_cargo'].sum()}")

Total de trabajadores: 1605


In [433]:


lista_cargos_importantes = [
    'DIRECTOR GENERAL',
    'JEFE DE SISTEMAS COMPUTACIONALES',
    'GERENTE COMERCIAL',
    'GERENTE DE CALIDAD DEL AGUA Y FISCALIZACION',
    'GERENTE DE SUPERVISION DE OBRA',
    'JEFE DE PROYECTOS',
    'GERENTE DE FINANZAS',
    'GERENTE DE PROYECTOS Y COSTOS',
    'SUBDIRECTORA DE PLANEACION',
    'ADMINISTRADOR DE REDES Y COMUNICACIONES',
    'GERENTE DE AGUA POTABLE Y ALCANTARILLADO',
    'PROGRAMADORA ANALISTA "A"',
    'PROGRAMADOR ANALISTA"B"',
    'PROGRAMADOR ANALISTA',
    'PROGRAMADORA ANALISTA',
    'JEFE DE TECNOLOGIAS DE LA OPERACION',
    'SUBDIRECTOR GENERAL OPERATIVO',
    'JEFE DE PLANEACION HIDRICA',
    'JEFE DE COMUNICACION',
    'JEFE DE COSTOS Y EVALUACION',
    'GERENTE DE TECNOLOGIAS DE LA INFORMACION Y COMUNICACION',
    'GERENTE SERVICIOS ADMINISTRATIVOS'
]


cargo_encontrados = []

for cargo in df_cargos_unicos:
    if cargo  in lista_cargos_importantes:
        cargo_encontrados.append(cargo)
if len(cargo_encontrados) == len(lista_cargos_importantes):
    print(f"Se encontraron todos los cargos importantes: {len(lista_cargos_importantes)}/{len(cargo_encontrados)}")



Se encontraron todos los cargos importantes: 22/22


In [434]:
df_trabajadores_de_interes = (
    df_filtrado
    .filter(
        pl.col("cargo").is_in(cargo_encontrados)
    )
    .sort("salario_bruto_mensual", descending=True)
    .collect()
)



In [435]:
df_trabajadores_de_interes.shape

(30, 2)

In [436]:
for cargo,sueldo in zip(
    df_trabajadores_de_interes["cargo"],
    df_trabajadores_de_interes["salario_bruto_mensual"]
):
    print(f"{cargo:>60}: ${sueldo:,.0f}")


                                            DIRECTOR GENERAL: $141,667
                               SUBDIRECTOR GENERAL OPERATIVO: $101,905
                                  SUBDIRECTORA DE PLANEACION: $90,120
                                           GERENTE COMERCIAL: $86,502
                 GERENTE DE CALIDAD DEL AGUA Y FISCALIZACION: $76,961
                              GERENTE DE SUPERVISION DE OBRA: $76,961
                                         GERENTE DE FINANZAS: $76,961
                               GERENTE DE PROYECTOS Y COSTOS: $76,961
                    GERENTE DE AGUA POTABLE Y ALCANTARILLADO: $76,961
     GERENTE DE TECNOLOGIAS DE LA INFORMACION Y COMUNICACION: $76,961
                           GERENTE SERVICIOS ADMINISTRATIVOS: $76,961
                                        JEFE DE COMUNICACION: $75,488
                            JEFE DE SISTEMAS COMPUTACIONALES: $50,493
                                           JEFE DE PROYECTOS: $50,493
                  

In [437]:

HORAS_SEMANA = 48  # horas laborales por semana por persona
HORAS_DECISION_DEFAULT = 30.0  # horas/mes dedicadas a toma de decisiones por persona (ajusta tú)
HORAS_DECISION_EQUIPO_MES = 30.0  # horas totales/mes del EQUIPO entero en modo decisión (top-down)

df = df_trabajadores_de_interes.clone()





sueldo_mensual_total = df["salario_bruto_mensual"].sum()
sueldo_semanal_total = sueldo_mensual_total / 4
costo_hora_equipo = sueldo_semanal_total / HORAS_SEMANA

costo_decisiones_mes_topdown = costo_hora_equipo * HORAS_DECISION_EQUIPO_MES



df = df.with_columns([
    (pl.col("salario_bruto_mensual") / 4).alias("sueldo_semanal"),
    (pl.col("salario_bruto_mensual") / 4 / HORAS_SEMANA).alias("sueldo_hora"), #
])
print(df.head(10))

costo_total_hora_decsision = df["sueldo_hora"].sum()
print(f"{costo_total_hora_decsision:,.2f}")
print(f"Costo hora en 3 dias de decision: ${costo_total_hora_decsision * (3 * 4*12)*5:,.2f}") # HORA X 3 DÍAS X 4 SEMANAS

shape: (10, 4)
┌─────────────────────────────────┬───────────────────────┬────────────────┬─────────────┐
│ cargo                           ┆ salario_bruto_mensual ┆ sueldo_semanal ┆ sueldo_hora │
│ ---                             ┆ ---                   ┆ ---            ┆ ---         │
│ str                             ┆ f64                   ┆ f64            ┆ f64         │
╞═════════════════════════════════╪═══════════════════════╪════════════════╪═════════════╡
│ DIRECTOR GENERAL                ┆ 141667.3              ┆ 35416.825      ┆ 737.850521  │
│ SUBDIRECTOR GENERAL OPERATIVO   ┆ 101905.4              ┆ 25476.35       ┆ 530.757292  │
│ SUBDIRECTORA DE PLANEACION      ┆ 90119.6               ┆ 22529.9        ┆ 469.372917  │
│ GERENTE COMERCIAL               ┆ 86502.0               ┆ 21625.5        ┆ 450.53125   │
│ GERENTE DE CALIDAD DEL AGUA Y … ┆ 76961.2               ┆ 19240.3        ┆ 400.839583  │
│ GERENTE DE SUPERVISION DE OBRA  ┆ 76961.2               ┆ 19240.3        